In [1]:
# For the SOTF

import pandas as pd
import numpy as np
import datetime
import os
import logging
import gc
logging.basicConfig(filename='BL_audience_for_SOTF_0726_20190708.log', level=logging.INFO)
logging.info('Started'+str(datetime.datetime.now()))

def recurisve_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)


os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/audience_Kaylar_SOTF_markets_20190708'

In [2]:
samplerows = None

lastdate = datetime.date(2019,6,29) # Recent Saturday
start_Sunday = lastdate-datetime.timedelta(days=52*7-1)

store_list=pd.read_excel("./7.26 Multiple Store Market Waves.xlsx",dtype=str)
store_list['Market']=store_list['Market'].replace("nan",np.nan)
store_list['Market']=store_list['Market'].fillna(method='ffill')
print(store_list.shape)


(66, 2)


In [3]:
# Up to 2019-03-30
# All item level data, weekly and the 1-time transfered historical data
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recurisve_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,2,9)]
print(historical_daily_df.shape)
print("Min_Date: "+str(historical_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(historical_daily_df['week_end_dt'].max()))

(27, 2)
Min_Date: 2018-08-11
Max_Date: 2019-02-09


In [4]:
new_daily_data_folder="/home/jian/BigLots/2019_by_weeks/"
new_daily_data_list=list(recurisve_file_gen(new_daily_data_folder))
new_daily_data_list=[x for x in new_daily_data_list if (".txt" in x) & ("DailySales" in x)]
new_daily_data_list=[x for x in new_daily_data_list if "hist" not in x]

new_daily_df=pd.DataFrame({"file_path":new_daily_data_list})

new_daily_df['week_end_dt']=new_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("2019_by_weeks/MediaStorm_")[1][:10])
new_daily_df['week_end_dt']=new_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
new_daily_df=new_daily_df[new_daily_df['week_end_dt']>historical_daily_df['week_end_dt'].max()]
new_daily_df=new_daily_df[new_daily_df['week_end_dt']<=datetime.date(2019,6,29)]
print(new_daily_df.shape)
print("Min_Date: "+str(new_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(new_daily_df['week_end_dt'].max()))

daily_df_file_after_20180922=historical_daily_df.append(new_daily_df)

new_dailysales_files=daily_df_file_after_20180922['file_path'].tolist()


(20, 2)
Min_Date: 2019-02-16
Max_Date: 2019-06-29


In [5]:
# >0 transactions only

combined_rewards_transaction=pd.DataFrame() 
count_i=1
print("Total Files: "+str(len(new_dailysales_files)))

for file_daily in new_dailysales_files:
    df=pd.read_table(file_daily,sep= '|',dtype =str,nrows=samplerows,
                     usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df[df['location_id']!="6990"]
    
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df[df['item_transaction_amt']>0]
    df_sales=df.groupby(['customer_id_hashed','transaction_dt','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates().groupby(['customer_id_hashed','transaction_dt','location_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_dt','location_id'],how="outer")
    df=df.rename(columns={"item_transaction_amt":"sales"})
    combined_rewards_transaction=combined_rewards_transaction.append(df)
    print(count_i,"done",datetime.datetime.now())
    count_i+=1
del df

# combined_rewards_transaction=combined_rewards_transaction.rename(columns={"item_transaction_amt":"sales"})

gc.collect()



Total Files: 47
1 done 2019-07-10 10:36:27.697562
2 done 2019-07-10 10:38:58.471671
3 done 2019-07-10 10:42:08.713859
4 done 2019-07-10 10:45:33.338688
5 done 2019-07-10 10:48:36.066649
6 done 2019-07-10 10:50:38.196644
7 done 2019-07-10 10:52:55.946503
8 done 2019-07-10 10:56:29.069668
9 done 2019-07-10 11:02:45.699516
10 done 2019-07-10 11:06:37.681216
11 done 2019-07-10 11:10:12.131824
12 done 2019-07-10 11:13:55.536308
13 done 2019-07-10 11:17:31.950436
14 done 2019-07-10 11:22:23.110595
15 done 2019-07-10 11:29:15.540918
16 done 2019-07-10 11:37:55.461263
17 done 2019-07-10 11:41:54.986219
18 done 2019-07-10 11:45:15.560666
19 done 2019-07-10 11:49:34.447416
20 done 2019-07-10 11:53:23.253112
21 done 2019-07-10 11:58:01.861451
22 done 2019-07-10 11:59:23.359220
23 done 2019-07-10 12:01:19.304394
24 done 2019-07-10 12:09:52.341956
25 done 2019-07-10 12:14:03.278500
26 done 2019-07-10 12:15:05.389110
27 done 2019-07-10 12:16:11.829143
28 done 2019-07-10 12:17:35.298514
29 done 2019-

175

In [6]:
daily_df_file_after_20180922['week_end_dt'].min()

datetime.date(2018, 8, 11)

In [7]:
start_Sunday

datetime.date(2018, 7, 1)

In [8]:
# Up to 2018-08-04
# All subclass level data, weekly and the 1-time transfered historical data
old_daily_data_folder="/home/jian/BigLots/2018_by_weeks/"
old_daily_data_list=list(recurisve_file_gen(old_daily_data_folder))
old_daily_data_list=[x for x in old_daily_data_list if (".txt" in x) & ("DailySales" in x)]

old_daily_df=pd.DataFrame({"file_path":old_daily_data_list})

old_daily_df['week_end_dt']=old_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("2018_by_weeks/MediaStorm_")[1][:10])
old_daily_df['week_end_dt']=old_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
old_daily_df=old_daily_df[old_daily_df['week_end_dt']>=start_Sunday]
old_daily_df=old_daily_df[old_daily_df['week_end_dt']<daily_df_file_after_20180922['week_end_dt'].min()]
old_daily_df.shape

(5, 2)

In [9]:
for file_daily_subclass in old_daily_df['file_path'].tolist():
    df=pd.read_table(file_daily_subclass,sep= '|',dtype =str,nrows=samplerows,
                     usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df[df['location_id']!="6990"]
    
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df[df['subclass_transaction_amt']>0]
    df_sales=df.groupby(['customer_id_hashed','transaction_dt','transaction_id','location_id'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates().groupby(['customer_id_hashed','transaction_dt','location_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_dt','location_id'],how="outer")
    df=df.rename(columns={'subclass_transaction_amt':"sales"})
    combined_rewards_transaction=combined_rewards_transaction.append(df)
    print(count_i,"done",datetime.datetime.now())
    count_i+=1
del df
gc.collect()

48 done 2019-07-10 12:47:33.903034
49 done 2019-07-10 12:48:59.122271
50 done 2019-07-10 12:50:13.630365
51 done 2019-07-10 12:51:26.453740
52 done 2019-07-10 12:52:49.418202


175

In [10]:
combined_rewards_transaction.shape

(76486016, 6)

In [11]:
combined_rewards_transaction['transaction_dt'].nunique()

364

In [12]:
agg_func={"trans":"sum","sales":"sum","transaction_dt":["min","max","nunique"]}
combined_rewards_transaction=combined_rewards_transaction.groupby(['customer_id_hashed','location_id'])['trans','sales','transaction_dt'].agg(agg_func).reset_index()

In [13]:
combined_rewards_transaction.head(2)

customer_id_hashed location_id trans  sales  \
                                                                   sum    sum   
0  00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...        1292     1  57.00   
1  00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...        5275     1  70.78   

  transaction_dt                      
             min         max nunique  
0     2018-10-30  2018-10-30       1  
1     2018-10-13  2018-10-13       1

In [14]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/audience_Kaylar_SOTF_markets_20190708'

In [15]:
cols=combined_rewards_transaction.columns.tolist()
new_cols=[]
for col in cols:
    if col[1]=="":
        new_col=col[0]
    else:
        new_col=col[0]+"_"+col[1]
    new_cols=new_cols+[new_col]
combined_rewards_transaction.columns=new_cols
combined_rewards_transaction.to_csv("./BL_recent_1_year_ids_by_store_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
combined_rewards_transaction.shape

(23208038, 7)

In [18]:
combined_rewards_transaction['customer_id_hashed'].nunique()

17939232

In [44]:
combined_rewards_transaction=pd.read_csv("./BL_recent_1_year_ids_by_store_JL_"+str(datetime.datetime.now().date())+".csv",dtype=str)
combined_rewards_transaction['trans_sum']=combined_rewards_transaction['trans_sum'].astype(int)

In [45]:
store_list=store_list.rename(columns={"Store ID":"location_id"})
combined_rewards_transaction=pd.merge(combined_rewards_transaction,store_list,on="location_id",how="left")
combined_rewards_transaction.head(5)

,customer_id_hashed,location_id,trans_sum,sales_sum,transaction_dt_min,transaction_dt_max,transaction_dt_nunique,Market
0,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,1292,1,57.0,2018-10-30,2018-10-30,1,NaN
1,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,5275,1,70.78,2018-10-13,2018-10-13,1,NaN
2,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,5309,1,84.0,2018-10-09,2018-10-09,1,NaN
3,000001dadc0265bf9d250566d74e0006323f18b5826641...,4061,3,143.25,2018-09-22,2018-12-21,3,NaN
4,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,5116,1,105.91000000000003,2018-11-12,2018-11-12,1,NaN


In [46]:
most_recent_store_visites=combined_rewards_transaction.sort_values(["customer_id_hashed",'transaction_dt_max'], ascending=[False,False])
most_recent_store_visites=most_recent_store_visites.drop_duplicates("customer_id_hashed")

id_total_trans=combined_rewards_transaction.groupby("customer_id_hashed")['trans_sum'].sum().to_frame().reset_index().rename(columns={"trans_sum":"total_transactions_for_id"})
combined_rewards_transaction=pd.merge(combined_rewards_transaction,id_total_trans,on="customer_id_hashed",how="outer")
combined_rewards_transaction['trans_pctg']=combined_rewards_transaction['trans_sum']/combined_rewards_transaction['total_transactions_for_id']

higher_50_df=combined_rewards_transaction[combined_rewards_transaction['trans_pctg']>=0.5]

higher_50_df_target_SOTF=higher_50_df[pd.notnull(higher_50_df['Market'])]
most_recent_store_visites_target_SOTF=most_recent_store_visites[pd.notnull(most_recent_store_visites['Market'])]
all_inclued_output=most_recent_store_visites_target_SOTF.append(higher_50_df_target_SOTF)
all_inclued_output=all_inclued_output[['customer_id_hashed','Market']].drop_duplicates()

In [54]:
all_inclued_output=all_inclued_output.rename(columns={"Market":"segment"})
all_inclued_output['segment']=all_inclued_output['segment']+"_0726SOTF"

In [60]:
all_inclued_output.to_csv("./BL_all_ids_for_the_5_mkts_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

# Append Emails

In [58]:
dfiddetail = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',nrows = samplerows)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')

dfiddetail=dfiddetail[['customer_id_hashed','email_address_hash']]

,customer_id_hashed,email_address_hash,customer_zip_code
0,9b7f9a1ba51dbae2a393c273ce8c9aa9bf610dc03410fd...,089bc373eda16a771e023d87ee368fb961c4ff9b36c7d6...,44402
1,c2c5bd862113c4e77c918dd22a25744f2e1224ea63b166...,065edf59d0ed26bea352be5f44fd7a6f688a0fa28cddb3...,37066


In [62]:
master_file_2018Q1_2019Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash'])


In [64]:
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash'])


In [69]:
all_master_files_after_201906=list(recurisve_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-06-01"]
df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash'])
    df_master_all_later_after_201906=df_master_all_later_after_201906.append(df)
df_master_all_later_after_201906.shape

(606131, 2)

In [70]:
dfiddetail=df_master_all_later_after_201906.append(master_file_gap).append(master_file_2018Q1_2019Q1).append(dfiddetail)
dfiddetail=dfiddetail.drop_duplicates("customer_id_hashed")

In [71]:
all_inclued_output=pd.merge(all_inclued_output,dfiddetail,on="customer_id_hashed",how="left")
all_inclued_output.shape

(885985, 14)

In [77]:
all_inclued_output=all_inclued_output[['customer_id_hashed','email_address_hash','segment']]

In [78]:
writer_folder="/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/audience_Kaylar_SOTF_markets_20190708/"+"output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)
all_inclued_output_non_naemail=all_inclued_output[pd.notnull(all_inclued_output['email_address_hash'])]

for mkt,df_group in all_inclued_output_non_naemail.groupby("segment"):
    df_group.to_csv(writer_folder+"BL_LR_segments_"+mkt.replace(" ","-")+"_"+str(datetime.datetime.now().date())+".csv",index=False)
    
    

In [79]:
summary_count_by_mkt_email=all_inclued_output_non_naemail.groupby("segment")['email_address_hash'].count().to_frame().reset_index()
summary_count_by_mkt_id=all_inclued_output.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()

summary_df_output=pd.merge(summary_count_by_mkt_email,summary_count_by_mkt_id,on="segment")
summary_df_output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/audience_Kaylar_SOTF_markets_20190708/summary_audience_df_0726_SOTF_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
